<a href="https://colab.research.google.com/github/imtheguna/PySpark-Learning/blob/GoogleColab/10_PySpark_Pivot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()
!pip install pyspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [830 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,798 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archiv

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
      .builder \
      .appName('SelectColumns').getOrCreate()

df = spark.read.csv('/content/data2.csv',header=True,inferSchema=True)

df = df.select('period','total_earnings','region_name')

df.show(3)

+-------+--------------+-----------+
| period|total_earnings|region_name|
+-------+--------------+-----------+
|2020.09|           953|  Northland|
|2020.09|         12420|   Auckland|
|2020.09|          3041|    Waikato|
+-------+--------------+-----------+
only showing top 3 rows



In [6]:
df1 = df.where(col('region_name').isin(['Gisborne','Canterbury'])).groupBy('period').pivot('region_name').sum('total_earnings')

df1.show(10)

+-------+----------+--------+
| period|Canterbury|Gisborne|
+-------+----------+--------+
|2022.12|      5113|     326|
|2022.03|      4607|     304|
|2020.12|      4266|     286|
|2021.03|      4196|     285|
|2021.06|      4341|     291|
|2021.12|      4734|     319|
|2022.09|      4900|     321|
|2022.06|      4762|     316|
|2021.09|      4450|     295|
|2020.09|      4232|     276|
+-------+----------+--------+



In [12]:
unpivotExpr = "stack(2, 'Gisborne',Gisborne,'Canterbury',Canterbury) as (region,total_earnings)"

unPivotDF = df1.select("period", expr(unpivotExpr))

unPivotDF.show()

+-------+----------+--------------+
| period|    region|total_earnings|
+-------+----------+--------------+
|2022.12|  Gisborne|           326|
|2022.12|Canterbury|          5113|
|2022.03|  Gisborne|           304|
|2022.03|Canterbury|          4607|
|2020.12|  Gisborne|           286|
|2020.12|Canterbury|          4266|
|2021.03|  Gisborne|           285|
|2021.03|Canterbury|          4196|
|2021.06|  Gisborne|           291|
|2021.06|Canterbury|          4341|
|2021.12|  Gisborne|           319|
|2021.12|Canterbury|          4734|
|2022.09|  Gisborne|           321|
|2022.09|Canterbury|          4900|
|2022.06|  Gisborne|           316|
|2022.06|Canterbury|          4762|
|2021.09|  Gisborne|           295|
|2021.09|Canterbury|          4450|
|2020.09|  Gisborne|           276|
|2020.09|Canterbury|          4232|
+-------+----------+--------------+

